In [1]:
%matplotlib qt
from matplotlib import pyplot as plt
from numpy.random import rand
import skimage as sm
import skimage.morphology as morph
from skimage.io import imread, imsave
from skimage.util import img_as_float, random_noise
from skimage.transform import rotate, resize
from skimage.filters import gaussian as ski_gaussian
from skimage.feature import canny, corner_harris, corner_peaks
from pylab import ginput
from scipy.signal import convolve, convolve2d, correlate2d, fftconvolve
from scipy.signal import  gaussian as scipy_gaussian
from scipy.fft import fft2, fftshift, ifft2, ifftshift
from scipy.interpolate import griddata
from scipy.ndimage import map_coordinates

import numpy as np
import os
import timeit


os.chdir("../Mats")

In [2]:
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300

# 1

1.1

In [27]:
A = imread("hand.tiff")
plt.imshow(A, cmap="gray"), plt.title("hand.tiff"), plt.axis("off")
plt.show()

(<matplotlib.image.AxesImage at 0x22ad1d25630>,
 Text(0.5, 1.0, 'hand.tiff'),
 (-0.5, 255.5, 255.5, -0.5))

In [31]:
B = canny(A, 1, 25, 100)
C = canny(A, 3, 25, 100)
D = canny(A, 1, 75, 100)
E = canny(A, 1, 25, 150)
plt.subplot(2, 2, 1)
plt.imshow(B, cmap="gray"), plt.title("\u03C3=1 low_t=25 high_t=100", fontsize=7), plt.axis("off")
plt.subplot(2, 2, 2)
plt.imshow(C, cmap="gray"), plt.title("\u03C3=3 low_t=25 high_t=100", fontsize=7), plt.axis("off")
plt.subplot(2, 2, 3)
plt.imshow(D, cmap="gray"), plt.title("\u03C3=1 low_t=75 high_t=100", fontsize=7), plt.axis("off")
plt.subplot(2, 2, 4)
plt.imshow(E, cmap="gray"), plt.title("\u03C3=1 low_t=25 high_t=150", fontsize=7), plt.axis("off")
plt.show()

1.2

In [56]:
A = imread("modelhouses.png")

k_A = corner_harris(A, "k", 0.05, sigma=1)
k_B = corner_harris(A, "k", 0.2, sigma=1)
k_C = corner_harris(A, "k", 0.05, sigma=3)
k_D = corner_harris(A, "k", 0.2, sigma=3)
plt.subplot(2, 2, 1)
plt.imshow(k_A, cmap="gray"), plt.title("k=0.05 \u03C3=1", fontsize=7), plt.axis("off")
plt.subplot(2, 2, 2)
plt.imshow(k_B, cmap="gray"), plt.title("k=0.2 \u03C3=1", fontsize=7), plt.axis("off")
plt.subplot(2, 2, 3)
plt.imshow(k_C, cmap="gray"), plt.title("k=0.05 \u03C3=3", fontsize=7), plt.axis("off")
plt.subplot(2, 2, 4)
plt.imshow(k_D, cmap="gray"), plt.title("k=0.2 \u03C3=3", fontsize=7), plt.axis("off")
plt.suptitle("Method k")
plt.show()

In [57]:
A = imread("modelhouses.png")
eps_A = corner_harris(A, "eps", 0.05, sigma=1)
eps_B = corner_harris(A, "eps", 0.2, sigma=1)
eps_C = corner_harris(A, "eps", 0.05, sigma=3)
eps_D = corner_harris(A, "eps", 0.2, sigma=3)
plt.subplot(2, 2, 1)
plt.imshow(eps_A, cmap="gray"), plt.title("eps=0.05 \u03C3=1", fontsize=7), plt.axis("off")
plt.subplot(2, 2, 2)
plt.imshow(eps_B, cmap="gray"), plt.title("eps=0.2 \u03C3=1", fontsize=7), plt.axis("off")
plt.subplot(2, 2, 3)
plt.imshow(eps_C, cmap="gray"), plt.title("eps=0.05 \u03C3=3", fontsize=7), plt.axis("off")
plt.subplot(2, 2, 4)
plt.imshow(eps_D, cmap="gray"), plt.title("eps=0.2 \u03C3=3", fontsize=7), plt.axis("off")
plt.suptitle("Method eps")
plt.show()

1.3

In [23]:
def find_corners(image, N_corners, method="k", k=0.05, eps=1e-06, sigma=1):
    image_corner = corner_harris(image, method, k, eps, sigma)
    return corner_peaks(image_corner, num_peaks=N_corners)

A = imread("modelhouses.png")
corners = find_corners(A, 250) 

plt.imshow(A, cmap="gray")
plt.scatter(corners[:,1], corners[:,0], s=0.5), plt.axis("off")
plt.show()


# 2

2.1

In [80]:

A  = imread("textlabel_gray_small.png")

corners = find_corners(A,50,k=0.05,sigma=7)

top_left = corners[np.argmin(np.sum(corners, axis=1))]
top_right = corners[np.argmin(np.diff(corners, axis=1))]
bottom_right = corners[np.argmax(np.sum(corners, axis=1))]
bottom_left = corners[np.argmax(np.diff(corners, axis=1))]
indices = []
for corner in [top_left, top_right, bottom_right, bottom_left]:
    index = np.where((corners == corner).all(axis=1))[0][0]
    indices.append(index)

corners = np.delete(corners, indices, axis=0)
top_left = corners[np.argmin(np.sum(corners, axis=1))]
top_right = corners[np.argmin(np.diff(corners, axis=1))]
bottom_right = corners[np.argmax(np.sum(corners, axis=1))]
bottom_left = corners[np.argmax(np.diff(corners, axis=1))]

plt.imshow(A,cmap="gray")
plt.scatter(corners[:,1], corners[:,0], s=0.5), plt.axis("off")

plt.show()



In [79]:
plt.imshow(A,cmap="gray")
plt.scatter(corners[:,1], corners[:,0], s=0.5), plt.axis("off")

plt.show()


In [75]:
top_left[0]

70

In [77]:
# Calculate the coordinates of the top_left and top_right corners
x_tl, y_tl = top_left
x_tr, y_tr = top_right

# Find the corners that are closest to the top_left and top_right corners in the x and y directions
x_closest = corners[np.argmin(np.abs(corners[:, 1] - x_tl)), :]
y_closest = corners[np.argmin(np.abs(corners[:, 0] - y_tr)), :]

# Calculate the Euclidean distances between the x_closest and y_closest corners
distances = np.linalg.norm(x_closest - y_closest)

# Get the coordinates of the corner closest to both x_closest and y_closest
closest_corner = corners[np.argmin(np.linalg.norm(corners - x_closest, axis=1) + np.linalg.norm(corners - y_closest, axis=1))]

In [76]:
x_tl,y_tl

(50, 70)

In [70]:
closest_corner

array([688,  57], dtype=int64)

In [78]:
plt.scatter(top_left[1],top_left[0], s=500), plt.axis("off")
plt.scatter(top_right[1],top_right[0], s=500), plt.axis("off")
plt.scatter(bottom_right[1],bottom_right[0], s=500), plt.axis("off")
plt.scatter(bottom_left[1],bottom_left[0], s=500), plt.axis("off")
plt.scatter(closest_corner[1],closest_corner[0], s=500), plt.axis("off")
plt.imshow(A,cmap="gray")


# 3

3.1

3.2

3.3

In [10]:
tau = np.linspace(-10, 10, 1000)
H = -tau**2/(np.pi*(4+tau**2))

plt.plot(tau, H)
plt.xlabel("\u03C4")
plt.ylabel("H(0, 0, \u03C4)")
plt.title("H(0, 0, \u03C4) plotted for various \u03C4")
plt.tight_layout()
plt.show()

i

ii

iii

3.4